# About this Example

This example demonstrates how to use Jupyter Notebooks to work with the Flowcean framework. 

![Turtlesim simulation](images/turtlesim.png)

This example is created using the package [turtlesim](http://wiki.ros.org/turtlesim), a tool designed for teaching the Robot Operating System (ROS). The example trains models to predict the next pose of the turtle, $\textbf{x}_{k+1}$, based on the current pose $\textbf{x}_{k}$ and velocity commands $\textbf{u}_{k}$.

![Motion model for turtlesim](images/turtlesim_model.svg)

ROS uses _topics_ to communicate between _nodes_. Nodes are processes that perform computations. They can **subscribe** to topics (receive messages from other nodes) or **publish** on topics (send messages to other nodes). In this example, the velocity commands are published to the `/turtle1/cmd_vel` topic by the turtle's teleop node (a node that lets you control the turtle interactively). The turtlesim node receives these velocity commands by subscribing to `/turtle1/cmd_vel` and publishes the turtle's pose to `/turtle1/pose`.

You can record ROS bag files using the _rosbag_ command-line tool. A ROS bag is a file format for storing ROS message data. It is commonly used for logging data during robot operation, which can later be played back for analysis or testing.
The tutorial uses ROS bag data recorded from turtlesim, processes it into supervised samples, trains multiple models, evaluates them using several metrics, and plots predictions versus ground truth.

For more information, please refer to the Jupyter notebooks provided in the `examples/jupyter_notebook/` directory.

**Topics used:**

- `/turtle1/cmd_vel` with fields `linear.x`, `angular.z`  
- `/turtle1/pose` with fields `x`, `y`, `theta`  

`/turtle1/cmd_vel` specifies the desired linear and angular velocities for the turtle. `/turtle1/pose` provides the turtle's actual position and orientation in the simulation environment.


## Section 0: Experiment Configuration

In [ ]:
import flowcean.cli

# The function below looks for a config.yaml in the current directory.
# In the config.yaml, we specify settings for our training run
config = flowcean.cli.initialize()

## Section 1 : Load and Prepare the Training Data
We will use the turtlesim example dataset for this tutorial. The dataset is a ROS 2 bag file that contains the pose and velocity of a turtle in the turtlesim simulator. The goal is to predict the next pose of the turtle given its current pose and velocity.

A config file specifies paths and parameters for the training run. This allows you to easily modify the training setup without changing the code and improves reproducibility. The contents of the config file can then be accessed in the code as a dictionary.  
See `config.yaml` for details of the configuration used in this workshop.  

The example expects two ROS bag directories in the config:

- Training: `rosbag.training_path`  
- Evaluation: `rosbag.evaluation_path`  


### Load Rosbags and Choose Inputs

First, we need to load the ROS 2 bag file and extract the relevant topics and fields. We will use the `from_rosbag` method of the `DataFrame` environment class.

The topics and fields we want to load are:   
```yaml  
  - /turtle1/cmd_vel
      - linear.x
      - angular.z
  - /turtle1/pose
      - x
      - y
      - theta
```

In [ ]:
from flowcean.polars import DataFrame

topics = {
    "/turtle1/cmd_vel": ["linear.x", "angular.z"],
    "/turtle1/pose": ["x", "y", "theta"],
}
# show current data structure without transforms
rosbag_train = DataFrame.from_rosbag(config.rosbag.training_path, topics=topics)
print(rosbag_train.observe().collect())


Note that the current data structure is nested, has only one line, and is not yet suitable for training. We will address this in the next tasks.


### Create Training Data Frame

Now that we have loaded the ROS 2 bag file, we need to create tabular training data that contains the input features and the target features. The input features are the current pose and velocity of the turtle, and the target feature is the next pose of the turtle.

The following table shows how the values of the current pose vector $\textbf{x}_{k}$ are shifted to form the next pose vector $\textbf{x}_{k+1}$. The next pose vector is the supervised target we want to predict.

| current pose $ \textbf{x}_{k} $ | velocity command $ \textbf{u}_{k} $     | next pose $ \textbf{x}_{k+1} $   |
|----------------------|----------------------|----------------------  |
| [0, 0, 0]            | [1, 0]               | <span style="color:red">[5, 0, 0]</span>            |
| <span style="color:red">[5, 0, 0]</span>          | [0, 1]               | <span style="color:green">[5, 0, 2]</span>          |
| <span style="color:green">[5, 0, 2]</span>       | [5, 0]               | [7, 4, 2]            |
| ...                    | ...                | ...                    |
| [4, 2, 2]     | [0, 0]                | null                    |

Note that this results in a null value for the next pose vector in the last row, which is filtered out.
We will use the `ZeroOrderHold`, `ExplodeTimeSeries`, and `ShiftInTime` transforms from the `flowcean.polars` module to create the training data frame.
You can concatenate/chain transforms to a dataframe with the `|` operator.
We modify the `transforms` variable to create a chain of transforms with the following steps:
- Call the `ZeroOrderHold` Transform:
  - Our features are our topics
  - Name the new column **"measurements"**

- Use the `|` operator to chain the `ExplodeTimeSeries` Transform: 
  - Apply the `ExplodeTimeSeries` transform to the **"measurements"** column

- Again, use the `|` operator to chain the `ShiftInTime` Transform:
  - Apply the `ShiftInTime` transform to the **"measurements"** column
  - Shift the features `/turtle1/pose/x`, `/turtle1/pose/y`, `/turtle1/pose/theta` 
  - Shift by **1 step** and give the new columns the suffix **"_next"**




In [ ]:
from _helper_functions import ShiftInTime
from flowcean.polars import DataFrame, ExplodeTimeSeries, ZeroOrderHold

transforms = (
    ZeroOrderHold(
        features=[
            "/turtle1/cmd_vel",
            "/turtle1/pose",
        ],
        name="measurements",
    )
    | ExplodeTimeSeries("measurements")
    | ShiftInTime(
        features=["/turtle1/pose/x", "/turtle1/pose/y", "/turtle1/pose/theta"],
        steps=1,
        suffix="_next",
    )
)

training_environment = (
    DataFrame.from_rosbag(config.rosbag.training_path, topics=topics) | transforms
)
evaluation_environment = (
    DataFrame.from_rosbag(config.rosbag.evaluation_path, topics=topics) | transforms
)
print(training_environment.observe().collect())

Now we have tabular data with aligned timestamps and separate columns for each feature.

## Section 2 : Select Learners across Libraries

Now that we have our training and evaluation samples, we can select learners from different libraries. We will use a `RandomForestRegressor` and a `RegressionTree` from sklearn, a `MultilayerPerceptron` from PyTorch, and an `XGBoostRegressor` from XGBoost.

In [ ]:
inputs = [
    "/turtle1/pose/x",
    "/turtle1/pose/y",
    "/turtle1/pose/theta",
    "/turtle1/cmd_vel/linear.x",
    "/turtle1/cmd_vel/angular.z",
]
outputs = [
    "/turtle1/pose/x_next",
    "/turtle1/pose/y_next",
    "/turtle1/pose/theta_next",
]


### Task 2.1 Learner configuration
We will use the configurations defined in the `config.yaml` file to initialize our learners. The configurations are stored in the `config.learners` attribute.

We initialize a regression tree, a random forest, a multilayer perceptron, and a XGBoost learner:
   - pass the tree configuration to the regression tree
   - pass the forest configuration to the random forest
   - pass a multilayer perceptron instance to the lightning learner and pass both their respective configurations
   - the XGBoost learner does not need to be configured


In [ ]:
from flowcean.sklearn import RandomForestRegressorLearner, RegressionTree
from flowcean.torch import LightningLearner, MultilayerPerceptron
from flowcean.xgboost import XGBoostRegressorLearner

regression_tree = RegressionTree(max_leaf_nodes=config.training.tree.max_leaf_nodes)

random_forest = RandomForestRegressorLearner(
    n_estimators=config.training.forest.n_estimators,
    max_depth=config.training.forest.max_depth,
)

mlp = LightningLearner(
    module=MultilayerPerceptron(
        learning_rate=config.training.mlp.learning_rate,
        output_size=len(outputs),
    ),
    batch_size=config.training.mlp.batch_size,
    max_epochs=config.training.mlp.max_epochs,
    accelerator="cpu",
)

xgb = XGBoostRegressorLearner() # no parameters to pass here

### Prepare Sequential Learning
We want to train all of our models in a looped fassion. To do this, we need to create a list of our learners.


In [ ]:
learners = [
    regression_tree,
    random_forest,
    mlp,
    xgb,
]

## Training of the Models

We will now train our models by looping over the learners list we created in the previous section and apply the `learn_offline` strategy.



### Create a Sequential Learning Loop

We will now create a training loop that will train each learner on the training samples. We will store the trained models in a list.

We implement the training loop by:
  - calling the `learn_offline` function and pass the required parameters
  - appending the trained model to the models list

In [ ]:
from flowcean.core import learn_offline

models = []
for learner in learners:
    print(f"Training model: {learner.name}")
    model = learn_offline(
        training_environment,
        learner,
        inputs=inputs,
        outputs=outputs,
    )
    models.append(model)

## Evaluation and Model Comparison

We will now evaluate our trained models on the data of the evaluation environment. We will use the `evaluate_offline` function from the `flowcean.training` to get an evaluation report for all the models which is printed in a tabular format.

### Choose Metrics for Evaluation

We want to evaluate our models using different metrics. We will define a list of metrics that we want to use for evaluation. The metrics we want to use are:
- Mean Absolute Error
- Mean Squared Error
- Regression Score (R2Score)
- Mean Euclidean Distance

Note: The euclidean distance requires the features/columns it should be calculated on (`/turtle1/pose/x_next`, `/turtle1/pose/y_next`).


In [ ]:
from euclidean_distance import MeanEuclideanDistance
from flowcean.sklearn import MeanAbsoluteError, MeanSquaredError, R2Score

metrics = [
    MeanAbsoluteError(),
    MeanSquaredError(),
    R2Score(),
    MeanEuclideanDistance(
        features=["/turtle1/pose/x_next", "/turtle1/pose/y_next"],
    ),
]

### Create an Evaluation Loop

We will use the `evaluate_offline` strategy to evaluate each trained model on the evaluation samples. The resulting report object contains the results of all metrics for each model which can be displayed in a table using the `great_table()` method.

Note: the `great_table()` only works in Jupyter Notebooks. For Python scripts, you can use `report.pretty_print()` to print the report to the console.


In [ ]:
from flowcean.core import evaluate_offline

report = evaluate_offline(
    models,
    environment=evaluation_environment,
    metrics=metrics,
    inputs=inputs,
    outputs=outputs,
)
report.great_table()


###  Select a Model and Visualization  

We want to select the best model based on the evaluation reports we created in the previous task. We will also visualize the predictions of the best model against the ground truth using the `predictions_vs_ground_truth` function from the same module.




In [ ]:
from _helper_functions import plot_predictions_vs_ground_truth

best_model = models[3] # the XGBoost model is the best one and 4th in the list
print(f"Best model: {best_model.name}")

# Plots are saved under plots/
plot_predictions_vs_ground_truth(
    environment=evaluation_environment,
    input_names=inputs,
    output_names=outputs,
    models=models,
)

# save model to disk
best_model.save("model.fml")

Now you have successfully trained and evaluated multiple machine learning models using the Flowcean framework! You have selected the best model based on the evaluation metrics and visualized its predictions against the ground truth. The best model is exported and can be used for deployment in a ROS 2 environment. 